In [1]:
# Based on https://ourworldindata.org/temperature-anomaly

import polars as pl

source = "monthly-surface-temperature-anomalies-by-year.csv"

df = pl.read_csv(source)

df.head()

Entity,Code,Year,2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995,1994,1993,1992,1991,…,1986,1985,1984,1983,1982,1981,1980,1979,1978,1977,1976,1975,1974,1973,1972,1971,1970,1969,1968,1967,1966,1965,1964,1963,1962,1961,1960,1959,1958,1957,1956,1955,1954,1953,1952,1951,1950
str,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Afghanistan""","""AFG""",1,4.5435214,-3.092134,0.9206118,0.241868,-1.31705,1.8286359,2.2859297,0.800035,2.352311,2.0118496,0.809244,-0.446632,-1.712392,0.732586,2.6423626,0.526383,-5.595724,-0.868612,-1.029035,-1.75616,2.259391,2.2624545,1.3290614,-0.887282,0.6626827,0.8500663,-0.716356,0.3431558,-2.795859,-0.438765,0.348055,-3.033496,-0.162744,-1.284097,…,-0.936852,-0.686402,-2.030046,-0.965905,-0.44863,0.028736,-1.453542,0.423748,-1.722132,-2.988588,1.2070822,-2.097282,-1.80225,-4.060106,-1.680301,-1.478685,-0.37708,-2.479886,-1.404595,-1.551208,1.7563959,0.8349867,-6.809986,1.0487983,-1.526134,-0.300404,0.036898,-1.090323,1.8199116,-2.995696,-2.108415,-0.948158,-1.033167,-1.672471,-1.569123,-1.882813,-1.412153
"""Afghanistan""","""AFG""",2,0.1638943,3.3264227,1.3052105,4.8273587,2.019373,-0.791941,2.7621717,0.039433,1.8491012,2.5761275,-3.534254,0.8698952,-3.855689,-0.602316,1.1868201,1.6612648,-3.050954,1.240218,4.776168,-2.389922,2.0808487,1.340041,0.675977,0.0211277,-2.435787,1.6706411,-1.377178,-2.260562,0.480786,-0.288667,-2.275126,1.2330197,-1.399277,-2.22134,…,-1.259191,1.696664,-5.42939,-0.682242,-2.867547,0.326035,-0.893347,0.094181,-0.706551,-1.262008,-1.156723,-2.225442,-4.361961,1.357963,-7.830344,0.020735,0.362668,-2.33013,-2.949835,-0.203151,2.6618738,-1.586789,-0.719293,3.4873848,2.0245204,-3.115107,2.980669,-2.406145,-0.595671,-3.608585,-0.405302,1.1384413,-1.164836,1.1227769,-0.062417,-3.503427,-4.856657
"""Afghanistan""","""AFG""",3,null,3.5448666,3.923089,3.2173767,0.356146,-0.802544,4.2338343,0.46722,2.0126696,-0.771881,-0.864267,1.2383728,-1.576733,-0.312193,3.1246634,1.7864456,3.2716599,-0.969269,1.2788115,0.6042862,3.4715843,-0.472038,1.7123146,0.53944,-1.347914,-1.201774,-1.283251,-1.866011,-1.056571,-2.764266,0.20839,-3.170706,-3.843225,-2.003184,…,-3.743624,-0.204167,1.4653082,-3.623787,-3.118774,0.4372368,-1.183301,-3.008913,-2.06883,2.77876,-3.340016,-2.348943,0.281681,-0.8985,-1.978459,0.6575303,-2.002678,2.4178581,-0.610296,-2.036999,-1.425849,-2.362708,0.085676,-0.197246,0.368731,-0.649406,-3.748532,-0.617683,1.1558638,-2.126567,-1.470247,0.505514,-2.20288,-1.970175,-1.76444,-3.506213,-3.069457
"""Afghanistan""","""AFG""",4,null,1.2093296,4.1597967,2.1004534,-0.38097,0.770987,1.2533731,1.1206369,0.056688,0.8462248,-0.374265,-0.887686,0.097024,0.5763893,2.1428595,-2.248569,0.9031811,2.375267,0.573334,-1.425703,1.2180643,0.5090666,0.9406719,2.424924,2.8480263,-0.505338,0.831154,-1.906176,-0.979177,-1.444772,-2.823973,-1.118769,-3.078444,-2.314029,…,-1.464323,0.4451561,0.325586,-2.751223,-0.698634,0.5687542,1.7855539,1.2505846,0.970597,0.7024412,-1.939593,-2.157334,0.54723,0.533978,-1.832222,0.799525,0.6924839,-1.167008,-1.483201,-3.00823,-2.18493,-2.884284,-1.312133,-0.001528,-0.518386,-3.374025,-3.871001,0.684994,1.6450548,-4.803986,-1.085439,-2.64906,-0.912584,-1.980633,0.051411,-3.384946,-4.735033
"""Afghanistan""","""AFG""",5,null,0.322958,1.20751,1.1306744,0.12882,-0.293928,-0.654427,2.4563847,2.0443516,0.6161976,-0.69989,-0.076105,-0.526955,2.4635887,0.70846,-0.112286,1.7069588,0.444725,2.9454536,-2.746923,0.9137287,-2.588592,0.798599,3.25601,3.6140518,-0.616243,-0.65583,-2.535543,-2.222988,-1.309998,-0.527592,-0.511196,-3.908453,-2.110401,…,-0.512884,-0.619675,0.193775,-0.207905,-1.229809,0.104395,0.42679,-3.72725,0.4021759,-0

In [2]:
# Created a wide dataframe, plot within the query

year_cols = [i for i in df.columns if i not in ["Entity", "Code", "Year"]][::-1]
plot_cols = ["Year"] + year_cols

year_colours = ['#8f0000' if int(i) > 2020 else '#D3D3D3' for i in year_cols]
year_hover_colours = ['#ff0000' if int(i) > 2020 else '#8f8080' for i in year_cols]
year_line_width = [2 if int(i) > 2020 else 1 for i in year_cols]

xtick_labels = [(1, "Jan"), (2, "Feb"), (3, "Mar"), (4, "Apr"), (5, "May"), (6, "Jun"), 
                (7, "Jul"), (8, "Aug"), (9, "Sep"), (10, "Oct"), (11, "Nov"), (12, "Dec")]

(df
 .melt(id_vars="Year", value_vars=year_cols)
 .with_columns(value=pl.col("value") / 2)
 .pivot(index="Year", columns="variable", values="value", aggregate_function="mean")
 .rename({"Year": "Month"})
 .plot(kind="line", x="Month", y=year_cols, color=year_colours,
       line_width=year_line_width, 
       legend=False, 
       attr_labels=year_cols,
       hover_line_color=year_hover_colours,
       xticks=xtick_labels,
       yformatter="%f C",
       title="Monthly surface temperature anomalies by year, World")
)

:NdOverlay   [Variable]
   :Curve   [Month]   (value)

In [6]:
# Create a long dataframe, use the "by" attribute and plot with hvplot

import hvplot.polars

year_cols = [i for i in df.columns if i not in ["Entity", "Code", "Year"]][::-1]

year_colours = ['#8f0000' if int(i) > 2020 else '#D3D3D3' for i in year_cols]
year_hover_colours = ['#ff0000' if int(i) > 2020 else '#8f8080' for i in year_cols]
year_line_width = [2 if int(i) > 2020 else 1 for i in year_cols]

data = (df
        .melt(id_vars="Year", value_vars=year_cols)
        .with_columns(value=pl.col("value") / 2)
        .rename({"Year": "Month", "variable": "Year", "value": "Deviation"})
        .group_by(["Month", "Year"]).agg(pl.col("Deviation").mean())
        .sort(["Year", "Month"])
        )

xtick_labels = [(1, "Jan"), (2, "Feb"), (3, "Mar"), (4, "Apr"), (5, "May"), (6, "Jun"), 
                (7, "Jul"), (8, "Aug"), (9, "Sep"), (10, "Oct"), (11, "Nov"), (12, "Dec")]

data.hvplot.line(
            x="Month", y="Deviation", by="Year", 
            color=year_colours,
            line_width=year_line_width, 
            legend=False, 
            attr_labels=year_cols,
            hover_line_color=year_hover_colours, 
            yformatter="%f C", 
            xticks=xtick_labels,
            grid=True,
            title="Monthly surface temperature anomalies by year, World"
            )

:NdOverlay   [Year]
   :Curve   [Month]   (Deviation)

In [4]:
# Unable to set: subtitle, reference/footer, insert image, grid lines